In [1]:
import openfermion as of

# Set Hubbard model parameters
x_dim = 2
y_dim = 3
tunneling = 1.0
coulomb = 4.0
spinless=True

# Create Hubbard model Hamiltonian as FermionOperator
hubbard_model = of.fermi_hubbard(x_dim, y_dim, tunneling, coulomb, spinless=spinless)

print(hubbard_model)

4.0 [0^ 0 1^ 1] +
4.0 [0^ 0 2^ 2] +
-1.0 [0^ 1] +
-1.0 [0^ 2] +
-1.0 [0^ 4] +
-1.0 [1^ 0] +
4.0 [1^ 1 3^ 3] +
-1.0 [1^ 3] +
-1.0 [1^ 5] +
-1.0 [2^ 0] +
4.0 [2^ 2 3^ 3] +
4.0 [2^ 2 4^ 4] +
-1.0 [2^ 3] +
-1.0 [2^ 4] +
-1.0 [3^ 1] +
-1.0 [3^ 2] +
4.0 [3^ 3 5^ 5] +
-1.0 [3^ 5] +
-1.0 [4^ 0] +
-1.0 [4^ 2] +
4.0 [4^ 4 0^ 0] +
4.0 [4^ 4 5^ 5] +
-1.0 [4^ 5] +
-1.0 [5^ 1] +
-1.0 [5^ 3] +
-1.0 [5^ 4] +
4.0 [5^ 5 1^ 1]


In [2]:
from scipy.sparse.linalg import expm_multiply

# Create a random initial state
n_qubits = of.count_qubits(hubbard_model)
initial_state = of.haar_random_vector(2**n_qubits, seed=7)

# Convert Hamiltonian to sparse matrix
hubbard_sparse = of.get_sparse_operator(of.jordan_wigner(hubbard_model))

# Set evolution time
time = 1.0

# Apply exp(-i H t) to the state
exact_state = expm_multiply(-1j*hubbard_sparse*time, initial_state)
print(hubbard_sparse)
print(exact_state)

  (2, 1)	(-1+0j)
  (4, 1)	(-1+0j)
  (16, 1)	(-1+0j)
  (1, 2)	(-1+0j)
  (8, 2)	(-1+0j)
  (32, 2)	(-1+0j)
  (3, 3)	(4+0j)
  (6, 3)	(1+0j)
  (9, 3)	(-1+0j)
  (18, 3)	(1+0j)
  (33, 3)	(-1+0j)
  (1, 4)	(-1+0j)
  (8, 4)	(-1+0j)
  (16, 4)	(-1+0j)
  (5, 5)	(4+0j)
  (6, 5)	(-1+0j)
  (9, 5)	(-1+0j)
  (17, 5)	(-1+0j)
  (20, 5)	(1+0j)
  (3, 6)	(1+0j)
  (5, 6)	(-1+0j)
  (10, 6)	(-1+0j)
  (12, 6)	(1+0j)
  (18, 6)	(-1+0j)
  (36, 6)	(1+0j)
  :	:
  (58, 57)	(-1+0j)
  (60, 57)	(-1+0j)
  (43, 58)	(-1+0j)
  (46, 58)	(1+0j)
  (54, 58)	(-1+0j)
  (57, 58)	(-1+0j)
  (58, 58)	(16+0j)
  (47, 59)	(1+0j)
  (55, 59)	(-1+0j)
  (59, 59)	(24+0j)
  (62, 59)	(1+0j)
  (30, 60)	(1+0j)
  (45, 60)	(-1+0j)
  (54, 60)	(1+0j)
  (57, 60)	(-1+0j)
  (60, 60)	(16+0j)
  (31, 61)	(1+0j)
  (55, 61)	(1+0j)
  (61, 61)	(24+0j)
  (62, 61)	(-1+0j)
  (47, 62)	(1+0j)
  (59, 62)	(1+0j)
  (61, 62)	(-1+0j)
  (62, 62)	(24+0j)
  (63, 63)	(36+0j)
[ 0.15317782-0.03474181j  0.08380517-0.12890515j -0.04758803-0.10386608j
  0.15132779+0.06175547j  0

In [3]:
import cirq
import openfermioncirq as ofc
import numpy as np

# Convert Hamiltonian to a DiagonalCoulombHamiltonian instance
hubbard_hamiltonian = of.get_diagonal_coulomb_hamiltonian(hubbard_model)

# Initialize qubits
qubits = cirq.LineQubit.range(n_qubits)

# Create circuit
circuit = cirq.Circuit.from_ops(
    ofc.simulate_trotter(
        qubits, hubbard_hamiltonian, time,
        n_steps=10,
        order=0,
        algorithm=ofc.trotter.LINEAR_SWAP_NETWORK)
)

# Apply the circuit to the initial state
result = circuit.apply_unitary_effect_to_state(initial_state)

# Compute the fidelity with the final state from exact evolution
fidelity = abs(np.dot(exact_state, result.conj()))**2

print(fidelity)
print(circuit)

0.9732473269809888
0: ───XXYY──────────YXXY───@──────────×ᶠ──────────────────────────────────────────────────────────────────────────────────────────────────────────XXYY──────────YXXY───@──────────×ᶠ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────XXYY──────────YXXY───@──────────×ᶠ────────────────────────────────────────────────────────────────────────────────────────────────────────────Rz(-0.0π)───────────────────────────────────────────────────────────────────────XXYY──────────#2───────@──────────×ᶠ────────────────────────────────────────────────────────────────────────────────────────────────────────XXYY──────────#2───────@──────────×ᶠ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────XXYY──────────#2───────@──────────×ᶠ──────────────────────────────────────────────Rz(-0.0π)───────────────XXYY──────────YXXY───@──────────×ᶠ───────────────────────

In [4]:
# Create circuit
circuit = cirq.Circuit.from_ops(
    ofc.simulate_trotter(
        qubits, hubbard_hamiltonian, time,
        n_steps=1,
        order=0,
        algorithm=ofc.trotter.LINEAR_SWAP_NETWORK,
        omit_final_swaps=True),
    strategy=cirq.InsertStrategy.EARLIEST
)

cirq.DropNegligible().optimize_circuit(circuit)
#print(circuit.to_text_diagram(transpose=True))

#print(study.circuit)

In [6]:
xmon_circuit = cirq.google.optimized_for_xmon(circuit)

print(xmon_circuit.to_text_diagram(transpose=True))

0                      1                        2                      3                      4                      5
│                      │                        │                      │                      │                      │
PhasedX(-0.5)^0.5      │                        PhasedX(0.0784)^0.5    │                      PhasedX(-0.542)^0.5    │
│                      │                        │                      │                      │                      │
@──────────────────────@                        @──────────────────────@                      @──────────────────────@
│                      │                        │                      │                      │                      │
PhasedX(0.5)^0.5       PhasedX(-1.0)^(9/11)     PhasedX(-0.922)^0.5    PhasedX(-0.422)^(9/11) PhasedX(0.458)^0.5     PhasedX(0.958)^(9/11)
│                      │                        │                      │                      │                      │
@──────────────────────@    

In [7]:
two_qubit_gates = xmon_circuit.findall_operations_with_gate_type(cirq.TwoQubitGate)
print(len(list(two_qubit_gates)))

39


In [8]:
ansatz = ofc.SwapNetworkTrotterAnsatz(
    hubbard_hamiltonian,
    iterations=3)
print(ansatz.circuit.to_text_diagram(transpose=True))

0    1            2            3            4            5
│    │            │            │            │            │
XXYY─XXYY^T_0_1_0 XXYY─────────XXYY^T_2_3_0 XXYY─────────XXYY^T_4_5_0
│    │            │            │            │            │
@────@^V_0_1_0    @────────────@^V_2_3_0    @────────────@^V_4_5_0
│    │            │            │            │            │
×ᶠ───×ᶠ           ×ᶠ───────────×ᶠ           ×ᶠ───────────×ᶠ
│    │            │            │            │            │
│    ×ᶠ───────────×ᶠ           ×ᶠ───────────×ᶠ           │
│    │            │            │            │            │
XXYY─XXYY^T_1_3_0 ×ᶠ───────────×ᶠ           XXYY─────────XXYY^T_2_4_0
│    │            │            │            │            │
@────@^V_1_3_0    │            │            @────────────@^V_2_4_0
│    │            │            │            │            │
×ᶠ───×ᶠ           │            │            ×ᶠ───────────×ᶠ
│    │            │            │            │            │
│    XXYY───────

In [9]:
n_electrons = 2

# Compute the ground state energy at the specified number of electrons
hamiltonian_sparse = of.get_sparse_operator(hubbard_model)
true_ground_energy, _ = of.jw_get_ground_state_at_particle_number(
    hamiltonian_sparse, n_electrons)

print(true_ground_energy)

-3.1231056256176655


In [10]:
# Use preparation circuit for eigenstate of one-body term
preparation_circuit = cirq.Circuit.from_ops(
    ofc.prepare_gaussian_state(
        ansatz.qubits,
        of.QuadraticHamiltonian(hubbard_hamiltonian.one_body),
        occupied_orbitals=range(n_electrons)
    )
)

# Define the objective function
objective = ofc.HamiltonianObjective(hubbard_hamiltonian)

# Create a variational study
study = ofc.VariationalStudy(
    name='hubbard_study',
    ansatz=ansatz,
    objective=objective,
    preparation_circuit=preparation_circuit)

print("Created a variational study with {} parameters.".format(
    study.num_params)
)
print()

print("The energy of the default initial guess is {}.".format(
    study.value_of(ansatz.default_initial_params()))
)
print()

print("The initial guess is:")
print(ansatz.default_initial_params())
print(study.circuit)

Created a variational study with 54 parameters.

The energy of the default initial guess is 1.6533237715694717.

The initial guess is:
[ 0.18028137  0.72676046  0.18028137  0.72676046  0.18028137  0.72676046
  0.18028137  0.72676046  0.18028137  0.72676046  0.18028137  0.72676046
  0.18028137  0.72676046  0.18028137  0.72676046  0.18028137  0.72676046
  0.18028137  0.18028137  0.18028137  0.18028137  0.18028137  0.18028137
  0.18028137  0.18028137  0.18028137  0.18028137  0.18028137  0.18028137
  0.18028137  0.18028137  0.18028137  0.18028137  0.18028137  0.18028137
  0.18028137 -0.36619772  0.18028137 -0.36619772  0.18028137 -0.36619772
  0.18028137 -0.36619772  0.18028137 -0.36619772  0.18028137 -0.36619772
  0.18028137 -0.36619772  0.18028137 -0.36619772  0.18028137 -0.36619772]
0: ───X───────────YXXY─────────────────────────────────────────────────────────────────────XXYY───────────@───────────×ᶠ────────XXYY───────────@───────────×ᶠ───────────────────────────────────XXYY───────────

In [12]:
# Perform an optimization run.
algorithm = ofc.optimization.ScipyOptimizationAlgorithm(kwargs={'method': 'COBYLA'},uses_bounds=False)
optimization_params = ofc.optimization.OptimizationParams(algorithm=algorithm,initial_guess=ansatz.default_initial_params())
result = study.optimize(optimization_params)

print("Optimized energy: {}".format(result.optimal_value))
print()
print("Optimized parameters:")
print(result.optimal_parameters)

Optimized energy: -3.1127152833799894

Optimized parameters:
[ 1.17932369  1.13750266  0.12164771  0.71605615  0.15332463  1.71374496
  0.0792486   0.71955572  0.04646572  0.72377395  0.22948251  0.80239815
  0.20728894  0.65767962  0.18861785  0.52488867  0.24813285  0.86838428
  0.07738491  0.2541136   0.15308266  1.27984936  0.25344416  1.15446724
  0.22973052  1.55357227  0.17987775  0.10412313  0.13994922  0.20078247
  0.17000803 -0.29243389  0.19079566  1.16492821  0.18222245  0.24471473
  0.25419313 -0.33649571  0.21860921  0.20039305  0.21273508  0.46717413
  0.13890438  0.60556737  0.13899313  0.48335191  0.19375487 -0.39193371
  0.24224539  0.49783563  0.15574729 -0.26859125  0.18040941 -0.38458656]


In [13]:
import openfermionpyscf as ofpyscf

# Set molecule parameters
geometry = [('H', (0.0, 0.0, 0.0)), ('H', (0.0, 0.0, 0.7414))]
basis = 'sto-3g'
multiplicity = 1
charge = 0

# Perform electronic structure calculations and
# obtain Hamiltonian as an InteractionOperator
hamiltonian = ofpyscf.generate_molecular_hamiltonian(
    geometry, basis, multiplicity, charge)

# Compute and print ground state energy
ground_energy, _ = of.get_ground_state(
    of.get_sparse_operator(hamiltonian))
print(ground_energy)

-1.137270174660897


In [14]:
class MyAnsatz(ofc.VariationalAnsatz):

    def params(self):
        """The parameters of the ansatz."""
        return [cirq.Symbol('theta_0')]

    def operations(self, qubits):
        """Produce the operations of the ansatz circuit."""
        q0, q1, q2, q3 = qubits
        yield cirq.H(q0), cirq.H(q1), cirq.H(q2)
        yield cirq.XPowGate(exponent=-0.5).on(q3)

        yield cirq.CNOT(q0, q1), cirq.CNOT(q1, q2), cirq.CNOT(q2, q3)
        yield cirq.ZPowGate(exponent=cirq.Symbol('theta_0')).on(q3)
        yield cirq.CNOT(q2, q3), cirq.CNOT(q1, q2), cirq.CNOT(q0, q1)

        yield cirq.H(q0), cirq.H(q1), cirq.H(q2)
        yield cirq.XPowGate(exponent=0.5).on(q3)

    def _generate_qubits(self):
        """Produce qubits that can be used by the ansatz circuit."""
        return cirq.LineQubit.range(4)

In [35]:
ansatz = MyAnsatz()
print(ansatz.circuit)



0: ───H────────@───────────────────────────────────@───H───
               │                                   │
1: ───H────────X───@───────────────────────@───────X───H───
                   │                       │
2: ───H────────────X───@───────────────@───X───────H───────
                       │               │
3: ───X^-0.5───────────X───Z^theta_0───X───X^0.5───────────


In [ ]:
# Choose an initial guess
initial_guess = [0.01]
print('Value of initial guess: {}'.format(study.value_of(initial_guess)))

# Choose an algorithm
algorithm = ofc.optimization.ScipyOptimizationAlgorithm(
    kwargs={'method': 'L-BFGS-B'},
    options={'maxiter': 10})

# Create OptimizationParams object
optimization_params = ofc.optimization.OptimizationParams(
    algorithm=algorithm,
    initial_guess=initial_guess)

# Run optimization
result = study.optimize(optimization_params)
print('Optimized value: {}'.format(result.optimal_value))
